# Classical denoising
Here, we denoise and post-process test spectra using a range of classical denoising techniques.
We use denoise spectra using a range of parameterisations for each technique.

In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from BaselineRemoval import BaselineRemoval
from sklearn.decomposition import PCA, KernelPCA
from skimage.restoration import denoise_wavelet
import matplotlib.pyplot as plt

In [2]:
#Load in the test set of low SNR spectra, the corresponding high SNR ground truths, and the cycleGAN denoised data

network_pred=np.load('./epoch_20/network_denoised.npy')
network_pred_GT = np.load('./epoch_20/network_denoised_GT.npy')
network_pred_input = np.load('./epoch_20/network_input.npy')
# we duplicate the noisy test spectra so we can baseline correct/normalise them for plotting, 
# while using the originals to evaluate our denoising techniques
network_pred_input_baseline_norm = network_pred_input

# test spectra used for evaluation
test_spectra = np.squeeze(network_pred_input)

# We add a small constant so we can apply a baseline correction to test spectra that have 
#already been baseline corrected. 
network_pred_GT[np.where(network_pred_GT==0)]=0.0001
network_pred_input_baseline_norm[np.where(network_pred_input_baseline_norm==0)]=0.0001
network_pred[np.where(network_pred==0)] = 0.0001


In [ ]:
# baseline correct the test spectra and ground truths, 
# and normalise the spectra based on their max value

for i in range(np.shape(network_pred)[0]):
    baseObj=BaselineRemoval(network_pred_GT[i])
    Modpoly_output=baseObj.ModPoly(3)
    network_pred_GT[i] = Modpoly_output/np.max(Modpoly_output)
    
    baseObj=BaselineRemoval(network_pred[i])
    Modpoly_output=baseObj.ModPoly(3)
    network_pred[i] = Modpoly_output/np.max(Modpoly_output)
    
    baseObj=BaselineRemoval(network_pred_input_baseline_norm[i])
    Modpoly_output=baseObj.ModPoly(3)
    network_pred_input_baseline_norm[i] = Modpoly_output/np.max(Modpoly_output)

In [6]:
np.save('network_pred_GT_corrected_normalised',network_pred_GT)
np.save('network_pred_corrected_normalised',network_pred)
np.save('network_input_corrected_normalised',network_pred_input_baseline_norm)

# Savitsky-Golay smoothing

In [8]:
SG_spectra = []

window_lengths = range(5,80,5)

for window_length in window_lengths:
    smoothed_spectra = np.zeros(np.shape(test_spectra))
    for i in range(np.shape(test_spectra)[0]):
        smoothed_spectra[i] = signal.savgol_filter(test_spectra[i], window_length=window_length, polyorder=3, mode="nearest")

    # baseline correct/normalise smoothed spectrum
    for i in range(np.shape(smoothed_spectra)[0]):
        # adding a small constant to enable baseline correction
        smoothed_spectra[i][np.where(smoothed_spectra[i]==0)]=0.0001

        baseObj=BaselineRemoval(smoothed_spectra[i])
        mod=baseObj.ModPoly(3)
        smoothed_spectra[i]=mod/np.max(mod)
        
    SG_spectra.append(smoothed_spectra)
SG_params = window_lengths

# Wiener filter

In [ ]:
W_spectra = []

window_lengths = range(5,80,5)

for window_length in window_lengths:
    smoothed_spectra = np.zeros(np.shape(test_spectra))
    for i in range(np.shape(test_spectra)[0]):
        smoothed_spectra[i] = signal.wiener(test_spectra[i], mysize=window_length)

    # baseline correct/normalise smoothed spectrum
    for i in range(np.shape(smoothed_spectra)[0]):
        # adding a small constant to enable baseline correction
        smoothed_spectra[i][np.where(smoothed_spectra[i]==0)]=0.0001

        baseObj=BaselineRemoval(smoothed_spectra[i])
        mod=baseObj.ModPoly(3)
        smoothed_spectra[i]=mod/np.max(mod)
        
    W_spectra.append(smoothed_spectra)
W_params = window_lengths

# Wavelet denoising

In [18]:
wavelet_spectra = []
wavelet_levels = range(1,10,1)
wavelet_spectra_params = wavelet_levels
for wavelet_level in wavelet_levels:
    spec_denoise_w = np.zeros(np.shape(test_spectra))
    for i in range(np.shape(test_spectra)[0]):
        spec_denoise_w[i] = denoise_wavelet(test_spectra[i], method='BayesShrink', mode='soft', wavelet_levels=wavelet_level, wavelet='sym8', rescale_sigma='True')
        #baseline correct and normalise
    for i in range(np.shape(spec_denoise_w)[0]):
        # adding a small constant to enable baseline correction
        spec_denoise_w[i][np.where(spec_denoise_w[i]==0)]=0.0001
        baseObj=BaselineRemoval(spec_denoise_w[i])
        mod=baseObj.ModPoly(3)
        spec_denoise_w[i]=mod/np.max(mod)
    wavelet_spectra.append(spec_denoise_w)

# Save all classically denoised spectra + params

In [19]:
np.save('wavelet_spectra',wavelet_spectra)
# 
np.save('SG_spectra',SG_spectra)
np.save('W_spectra',W_spectra)

np.save('wavelet_spectra_params',wavelet_spectra_params)
# 
np.save('SG_spectra_params',SG_params)
np.save('W_spectra_params',W_params)


